In [2]:
# Project Overview

# 1. Collect the Data
    # Use the NumPy module to generate more than 1,500 random latitudes and longitudes.
    # Use the citipy module to list the nearest city to the latitudes and longitudes.
    # Use the OpenWeatherMap API to request the current weather data from each unique city in your list.
    # Parse the JSON data from the API request.
    # Collect the following data from the JSON file and add it to a DataFrame:
        # - City, country, and date
        # - Latitude and longitude
        # - Maximum temperature
        # - Humidity
        # - Cloudiness
        # - Wind speed

# 2. Exploratory Analysis with Visualization
    # Create scatter plots of the weather data for the following comparisons:
        # - Latitude versus temperature
        # - Latitude versus humidity
        # - Latitude versus cloudiness
        # - Latitude versus wind speed
    # Determine the correlations for the following weather data:
        # - Latitude and temperature
        # - Latitude and humidity
        # - Latitude and cloudiness
        # - Latitude and wind speed
    # Create a series of heatmaps using the Google Maps and Places API that showcases the following:
        # - Latitude and temperature
        # - Latitude and humidity
        # - Latitude and cloudiness
        # - Latitude and wind speed

# 3. Visualize Travel Data
    # Create a heatmap with pop-up markers that can display information on specific cities based on a customer’s 
    # travel preferences. Complete these steps:
        # 1. Filter the Pandas DataFrame based on user inputs for a minimum and maximum temperature.
        # 2. Create a heatmap for the new DataFrame.
        # 3. Find a hotel from the cities’ coordinates using Google’s Maps and Places API, and Search Nearby feature.
        # 4. Store the name of the first hotel in the DataFrame.
        # 5. Add pop-up markers to the heatmap that display information about the city, current maximum temperature, 
        #    and a hotel in the city.

In [9]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [10]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

# Pack the latitudes (lats) and longitudes (lngs) as pairs by zipping them (lat_lngs) with the zip() function.
lat_lngs = zip(lats, lngs) 
lat_lngs 

In [11]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [13]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

# 1. We create a cities list to store city names.
# 2. We iterate through the coordinates, as in our practice, 
#    and retrieve the nearest city using the latitude and longitude pair.
# 3. We add a decision statement with the logical operator not in to determine whether the found city is already
#    in the cities list. If not, then we’ll use the append()function to add it. We are doing this because among 
#    the 1,500 latitudes and longitudes, there might be duplicates, which will retrieve duplicate cities, 
#    and we want to be sure we capture only the unique cities.

608